# $R_t$ and $k_t$ Negative-Binomial Plots of Number of Secondary Cases

This notebook contains the code for plotting the distribution of number of secondary cases along with their Negative-Binomial fits. As such, the code still requires a bit of manual adaptation for each experiment. So the code here corresponds to the code to generate the plots in the paper.


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
import itertools
import numpy as np
from scipy import stats as sps
import scipy.optimize
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from lib.calibrationSettings import (
    calibration_lockdown_dates, 
    calibration_states,
)
from lib.experiment import load_summary
from lib import plot as lib_plot
from lib import rt_nbinom  # Module for NB fit of Rt/kt

---

## Experiment in continued lockdown

In [ ]:
def plot_daily_rt(country, area, num_weeks, xtick_interval, plot_rt=True, plot_pmf=True):
    print(f"{country}-{area}: {num_weeks}")
    print('-'*20)
    # Load the experiment summary
    summary_name = f'continued-lockdown-{country}-{area}-extended_lockdown_weeks={num_weeks}.pk'
    result = load_summary(summary_name)
    # Extract the lockdown start time for the country
    country_code = result.metadata.country
    lockdown = calibration_lockdown_dates[country_code]
    print((f"{country_code}: simulation starts on {result.metadata.start_date}, "
           f"lockdown starts on {lockdown['start']}"))
    # Plot daily Rt over time
    if plot_rt:
        print('-- Plot daily Rt over time...')
        plotter = lib_plot.Plotter()
        plotter.plot_daily_nbinom_rts(
            result=result,
            slider_size=24.0, window_size=24.*5, end_cutoff=24.*7,
            ymax=4.4,
#             lockdown_label='Interventions', 
#             lockdown_label_y=2.7,
#             figsize=(2.65, 1.9),
            lockdown_label='Interv.', 
            lockdown_label_y=3.2,
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV,
            lockdown_at=pd.to_datetime(lockdown['start']), 
            lockdown_xshift=1.1 * result.summary.max_time / 1560.0,
            xtick_interval=xtick_interval[country][area],
            figformat='neurips-double',
            filename=f"{result.metadata.experiment_info}_{result.metadata.simulation_info}-daily-Rt"
    )
    # Plot 2ndary cases distributions
    if plot_pmf:
        print('-- Plot 2ndary cases distributions...')
        if num_weeks == 100:
            t0_range=[0.0 * 24,   # the week before lockdown
                      15.0 * 24,  # right after start of lockdown
                     ]
            window_size=10.0*24
            label_range=['pre-lockdown',
                         'in-lockdown'
                        ]
        else:
            t0_range=[0.0 * 24,   # the week before lockdown
                      11.0 * 24,  # right after start of lockdown
                      38.0 * 24   # right after end of lockdown
                     ]
            window_size=7.0*24
            label_range=['pre-lockdown',
                         'in-lockdown',
                         'post-lockdown'
                        ]
        plotter = lib_plot.Plotter()
        plotter.plot_nbinom_distributions(
            result=result,
            x_range=np.arange(0, 20),
            t0_range=t0_range,
            window_size=window_size,
            label_range=label_range,
            ymax=0.85,
#             figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE_TALL,
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV,
            figformat='neurips-double',
            filename=f"{result.metadata.experiment_info}_{result.metadata.simulation_info}-pmf"
        )
    print()


places = [
    ('CH', 'TI'),
    ('CH', 'BE'), 
    ('CH', 'JU'), 
    ('GER', 'KL'),  
    ('GER', 'RH'), 
    ('GER', 'TU')
]

xtick_interval = {
    'GER': {'RH': 2, 'TU': 2, 'KL': 2},
    'CH': {'BE': 2, 'JU': 2, 'TI': 2},
}

for country, area in places:
    for num_weeks in [4, 100]:
        plot_daily_rt(country, area, num_weeks, xtick_interval, plot_rt=True, plot_pmf=False)



In [ ]:
# Crop pdf margins
!bash crop_pdfs.sh plots/daily-nbinom-rts-*.pdf
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf

---

Custom plot parameter values for GER-RH

In [ ]:
summary_name = 'continued-lockdown-GER-RH-extended_lockdown_weeks=4.pk'

# Load the experiment summary
result = load_summary(summary_name)

# Extract the lockdown start time for the country
country_code = result.metadata.country
lockdown = calibration_lockdown_dates[country_code]
print((f"{country_code}: simulation starts on {result.metadata.start_date}, "
       f"lockdown starts on {lockdown['start']}"))

plotter = lib_plot.Plotter()

In [ ]:
t0_range = [
    1.0 * 24,   # the week before lockdown
    15.0 * 24,  # right after start of lockdown
    38.0 * 24   # right after end of lockdown
]
window_size = 10.0*24
label_range = [
    'pre-lockdown',
    'in-lockdown',
    'post-lockdown'
]
x_range = np.arange(0, 20)

In [ ]:
interval_range = [(t0, t0 + window_size) for t0 in t0_range]
df = plotter._compute_nbinom_distributions(result=result, x_range=x_range, interval_range=interval_range)

In [ ]:
plotter = lib_plot.Plotter()
plotter.plot_nbinom_distributions(
    result=result,
    df=df,
    x_range=x_range,
    t0_range=t0_range,
    window_size=window_size,
    label_range=label_range,
    ymax=0.85,
    figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE_TALL,
    figformat='neurips-double',
    filename=f"{result.metadata.experiment_info}_{result.metadata.simulation_info}-pmf"
)

# Crop pdf margins
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf

## During contact tracing

In [ ]:
def plot_daily_rt_tracing(country, area, plot_rt=True, plot_pmf=True):
    print(f"{country}-{area}:")
    print('-'*20)
    # Load the experiment summary
    summary_name = f'tracing-{country}-{area}-tracing=isolate+test-'\
                  'delay=48.0-contacts_tested=100000-test_policy=basic.pk'
        
    result = load_summary(summary_name)
    # Extract the lockdown start time for the country
    country_code = result.metadata.country
    lockdown = calibration_lockdown_dates[country_code]
    print((f"{country_code}: simulation starts on {result.metadata.start_date}, "
           f"lockdown starts on {lockdown['start']}"))
    # Plot daily Rt over time
    if plot_rt:
        print('-- Plot daily Rt over time...')
        plotter = lib_plot.Plotter()
        df = plotter._estimate_daily_nbinom_rts(result, slider_size=24.0, window_size=24.*7, end_cutoff=24.*10)
        
        plotter.plot_daily_nbinom_rts(
            result=result, 
            filename=f"{result.metadata.experiment_info}-daily-Rt",
            df = df, # <<<<<<<<<<<<
            slider_size=24.0, window_size=24.*7, end_cutoff=24.*10,
            cmap_range=(0.5, 1.5),
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV, 
            figformat='neurips-double',
            ymax=4.4,
            xlim=(0, 127),
            x_axis_dates=False,
            subplots_adjust={'bottom':0.2, 'top': 0.98, 'left': 0.12, 'right': 0.96},
        )        
        
    # Plot 2ndary cases distributions
    if plot_pmf:
        print('-- Plot 2ndary cases distributions...')
        
        t0_range = [50 * 24.0] # after 50 days
        window_size = 10.0*24
        label_range = ['during-tracing']
        
        plotter = lib_plot.Plotter()
        plotter.plot_nbinom_distributions(
            result=result,
            x_range=np.arange(0, 20),
            t0_range=t0_range,
            window_size=window_size,
            label_range=label_range,
            ymax=0.85,
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV,
            figformat='neurips-double',
            filename=f"{result.metadata.experiment_info}-pmf"
        )
    print()

        
        
plot_daily_rt_tracing('CH', 'TI', plot_rt=True, plot_pmf=True)
plot_daily_rt_tracing('CH', 'BE', plot_rt=True, plot_pmf=True)
plot_daily_rt_tracing('CH', 'JU', plot_rt=True, plot_pmf=True)
plot_daily_rt_tracing('GER', 'TU', plot_rt=True, plot_pmf=True)
plot_daily_rt_tracing('GER', 'KL', plot_rt=True, plot_pmf=True)
plot_daily_rt_tracing('GER', 'RH', plot_rt=True, plot_pmf=True)


# Crop pdf margins
!bash crop_pdfs.sh plots/daily-nbinom-rts-*.pdf
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf

## K groups 

In [ ]:
def plot_daily_rt_kgroups(country, area, plot_rt=True, plot_pmf=True):
    print(f"{country}-{area}:")
    print('-'*20)
    # Load the experiment summary
    summary_name = f'k-groups-{country}-{area}-K_groups=2.pk'
        
    result = load_summary(summary_name)
    # Extract the lockdown start time for the country
    country_code = result.metadata.country
    lockdown = calibration_lockdown_dates[country_code]
    print((f"{country_code}: simulation starts on {result.metadata.start_date}, "
           f"lockdown starts on {lockdown['start']}"))
    # Plot daily Rt over time
    if plot_rt:
        print('-- Plot daily Rt over time...')
        plotter = lib_plot.Plotter()
        df = plotter._estimate_daily_nbinom_rts(result, slider_size=24.0, window_size=24.*7, end_cutoff=24.*10)
        
        plotter.plot_daily_nbinom_rts(
            result=result, 
            filename=f"{result.metadata.experiment_info}-daily-Rt",
            df = df, # <<<<<<<<<<<<
            slider_size=24.0, window_size=24.*7, end_cutoff=24.*10,
            cmap_range=(0.5, 1.5),
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV, 
            figformat='neurips-double',
            lockdown_label='Interv.', 
            lockdown_label_y=3.2,
            lockdown_at=pd.to_datetime(lockdown['start']), 
            lockdown_xshift=1.1 * result.summary.max_time / 1560.0,
            ymax=4.4,
        )        
            
    # Plot 2ndary cases distributions
    if plot_pmf:
        print('-- Plot 2ndary cases distributions...')
        
        t0_range = [50 * 24.0] # after 50 days
        window_size = 10.0*24
        label_range = ['during-tracing']
        
        plotter = lib_plot.Plotter()
        plotter.plot_nbinom_distributions(
            result=result,
            x_range=np.arange(0, 20),
            t0_range=t0_range,
            window_size=window_size,
            label_range=label_range,
            ymax=0.85,
            figsize=lib_plot.FIG_SIZE_FULL_PAGE_TRIPLE_ARXIV,
            figformat='neurips-double',
            filename=f"{result.metadata.experiment_info}-pmf"
        )
    print()

        
        
plot_daily_rt_kgroups('CH', 'TI', plot_rt=True, plot_pmf=True)
plot_daily_rt_kgroups('CH', 'BE', plot_rt=True, plot_pmf=True)
plot_daily_rt_kgroups('CH', 'JU', plot_rt=True, plot_pmf=True)
plot_daily_rt_kgroups('GER', 'TU', plot_rt=True, plot_pmf=True)
plot_daily_rt_kgroups('GER', 'KL', plot_rt=True, plot_pmf=True)
plot_daily_rt_kgroups('GER', 'RH', plot_rt=True, plot_pmf=True)


# Crop pdf margins
!bash crop_pdfs.sh plots/daily-nbinom-rts-*.pdf
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf